In [1]:
# === Quick responsiveness test inside a notebook ===
# Prereqs:
#   pip install ollama pandas
from pathlib import Path
import json, re
import pandas as pd
from ollama import chat   # matches your example
# ---------------- Configuration ----------------
CASE_BACKGROUND = """
This is a hypothetical antitrust investigation about widget pricing.
Documents are responsive if they discuss widget pricing strategy,
communications with competitors, or evidence of market allocation.
"""
FOLDER = "./test_docs"  # folder containing files to test
MODELS = ["qwen3:4b","gemma3n:e4b",  "llama3.2:3b"]
SYSTEM_PROMPT = (
   "You are an eDiscovery reviewer. "
   "Decide if the document is RESPONSIVE to the case background. "
   "Output JSON: {\"label\":\"Responsive|Not Responsive|Needs Review\", \"rationale\":\"...\"}"
)
USER_TEMPLATE = "CASE BACKGROUND:\n{case}\n\nDOCUMENT:\n{doc}"
def read_file_text(p: Path, max_chars=5000) -> str:
   try:
       return p.read_text(encoding="utf-8", errors="ignore")[:max_chars]
   except:
       return ""
def call_model(model: str, case_text: str, doc_text: str) -> dict:
   msg = USER_TEMPLATE.format(case=case_text, doc=doc_text)
   resp = chat(
       model=model,
       messages=[
           {"role": "system", "content": SYSTEM_PROMPT},
           {"role": "user", "content": msg},
       ],
       think=False
   )
   raw = resp.message.content
   m = re.search(r"\{.*\}", raw, re.DOTALL)
   try:
       data = json.loads(m.group(0) if m else raw)
       label = str(data.get("label","Needs Review")).strip()
       rationale = str(data.get("rationale","")).strip()
   except Exception:
       label, rationale = "Needs Review", f"Parse error: {raw[:100]}"
   return {"label": label, "rationale": rationale}
def majority_vote(labels):
   votes = {"Responsive":0, "Not Responsive":0, "Needs Review":0}
   for l in labels:
       if l in votes: votes[l]+=1
       else: votes["Needs Review"]+=1
   best = max(votes.items(), key=lambda kv: kv[1])
   # tie -> Needs Review
   if list(votes.values()).count(best[1]) > 1:
       return "Needs Review"
   return best[0]
results = []
for f in Path(FOLDER).glob("*"):
   print(f)
   if not f.is_file(): continue
   text = read_file_text(f)
   model_votes, model_rats = {}, {}
   for m in MODELS:
       out = call_model(m, CASE_BACKGROUND, text)
       model_votes[m] = out["label"]
       model_rats[m] = out["rationale"]
   final = majority_vote(list(model_votes.values()))
   print(final)
   results.append({
       "file": f.name,
       "final_vote": final,
       **{f"vote_{m}": v for m,v in model_votes.items()},
       **{f"rat_{m}": r for m,r in model_rats.items()},
   })
df = pd.DataFrame(results)
df

test_docs\New Text Document (2).txt
Not Responsive
test_docs\New Text Document (3).txt
Needs Review
test_docs\New Text Document (4).txt
Not Responsive
test_docs\New Text Document.txt
Responsive


,file,final_vote,vote_qwen3:4b,vote_gemma3n:e4b,vote_llama3.2:3b,rat_qwen3:4b,rat_gemma3n:e4b,rat_llama3.2:3b
0,New Text Document (2).txt,Not Responsive,Needs Review,Not Responsive,Not Responsive,Parse error: <think>\nWe are given a case back...,This document is an invitation to a holiday pa...,The document does not discuss widget pricing s...
1,New Text Document (3).txt,Needs Review,Needs Review,Responsive,Not Responsive,Parse error: <think>\nWe are given a case back...,"The document explicitly discusses ""Regional Ma...",The document does not discuss widget pricing s...
2,New Text Document (4).txt,Not Responsive,Needs Review,Not Responsive,Not Responsive,Parse error: <think>\nWe are given a case back...,"The document discusses employee spotlights, vo...",The document does not mention widget pricing s...
3,New Text Document.txt,Responsive,Needs Review,Responsive,Responsive,Parse error: <think>\nWe are given a case back...,The document directly discusses widget pricing...,The document discusses widget pricing strategy...


In [1]:
import ollama
print(ollama.list())

ModuleNotFoundError: No module named 'ollama'

In [ ]:
# --- Call Responsiveness API (single model) on local files ---
# Prereqs:
#   1) API running:  uvicorn api.main:app --reload --port 8088
#   2) Folder: ./test_docs with the sample .txt files we made earlier
#   3) pip install requests  (if not already)
import os, json, csv, textwrap, requests
from pathlib import Path
API_URL = "http://localhost:8088/review"
MODEL_NAME = "gemma3n:e4b"           # <-- change if you want a different single model
DOC_FOLDER = Path("./test_docs")     # <-- folder with your docs
OUT_CSV = Path("./results_single.csv")
CASE_BACKGROUND = """Antitrust investigation about widget pricing and market allocation.
Documents are responsive if they discuss widget pricing strategy, communications with competitors,
or evidence of regional market allocation or non-solicitation agreements.
"""
# 1) Collect documents (simple .txt scan)
docs = []
if DOC_FOLDER.exists():
   for p in sorted(DOC_FOLDER.glob("*.txt")):
       try:
           txt = p.read_text(encoding="utf-8", errors="ignore")[:20000]
           docs.append({"filename": p.name, "text": txt})
       except Exception as e:
           print(f"Skipping {p.name}: {e}")
else:
   raise SystemExit(f"Folder not found: {DOC_FOLDER.resolve()}  (create it and add .txt files)")
if not docs:
   raise SystemExit("No .txt files found in test_docs. Add a few and re-run.")
# 2) Build request payload (single model mode)
payload = {
   "mode": "single",
   "model_name": MODEL_NAME,
   "case_background": CASE_BACKGROUND,
   "docs": docs
}
# 3) POST to API
try:
   resp = requests.post(API_URL, json=payload, timeout=120)
   resp.raise_for_status()
except requests.RequestException as e:
   raise SystemExit(f"API request failed: {e}\nIs the server running at {API_URL}?")
data = resp.json()
# 4) Pretty print minimal table (no pandas)
rows = []
for r in data.get("results", []):
   fn = r["filename"]
   final = r["final_label"]
   # since mode=single, votes/rationales has one key (the model)
   model_key = next(iter(r["votes"].keys()))
   vote = r["votes"][model_key]
   rationale = r["rationales"].get(model_key, "")
   rows.append((fn, final, model_key, vote, rationale))
# column widths
headers = ("file", "final", "model", "model_vote", "rationale")
widths = [max(len(h), *(len(str(row[i])) for row in rows)) for i, h in enumerate(headers)]
def line(parts, widths):
   return " | ".join(str(p).ljust(w) for p, w in zip(parts, widths))
print(line(headers, widths))
print("-+-".join("-"*w for w in widths))
for row in rows:
   # wrap rationale to keep the console tidy
   wrapped = textwrap.wrap(row[4], width=80) or [""]
   print(line(row[:4] + (wrapped[0],), widths))
   for cont in wrapped[1:]:
       print(line(("", "", "", "", cont), widths))
# 5) Save CSV
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
with OUT_CSV.open("w", newline="", encoding="utf-8") as f:
   w = csv.writer(f)
   w.writerow(headers)
   for row in rows:
       w.writerow(row)
print(f"\nSaved CSV: {OUT_CSV.resolve()}")